In \[38\]:

    from camel_tools.sentiment import SentimentAnalyzer
    from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline
    import torch
    import numpy as np
    import re
    import pandas as pd

In \[39\]:

    tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
    model =AutoModelForSequenceClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
    #snt = SentimentAnalyzer("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
    #sa = pipeline('text-classification', model='CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment')

In \[110\]:

    #Arabic Preprocessing
    class Arabic_Preprocessing:
        
        def cleaning_arabic_content(self,text):  
            if text is None:
                text=""
                return text
            search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
                  "\\",'\n', '\t','&quot;','?','؟','!']
            replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
                       "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
            #remove tashkeel
            tashkeel = re.compile(r'[\u0617-\u0618-\u0619-\u061A\u064B-\u0652-\u064C-\u064D-\u064E-\u064F-\u0650-\u0651-\u0652-\u0653-\u0654-\u0655]')
            text = re.sub(tashkeel,"", text)

            longation = re.compile(r'(.)\1+')
            subst = r"\1\1"
            
            text = re.sub(longation, subst, text)

            text = re.sub(r"[^\w\s]", '', text)
            
            #remove english words
            text = re.sub(r"[a-zA-Z]", '', text)
            
            #remove spaces
            text = re.sub(r"\d+", ' ', text)
            text = re.sub(r"\n+", ' ', text)
            text = re.sub(r"\t+", ' ', text)
            text = re.sub(r"\r+", ' ', text)
            text = re.sub(r"\s+", ' ', text)
            
            #remove repetetions
            text = text.replace('وو', 'و')
            text = text.replace('يي', 'ي')
            text = text.replace('اا', 'ا')

            for i in range(0, len(search)):
                text = text.replace(search[i], replace[i])

            text = text.strip()

            return text

    #Arabic Sentiment Analysis Class
    class Arabic_Sentiment:
        
        def arabic_sentiment_analysis(self,content):
            pre=Arabic_Preprocessing()
            #Using Camel Tools
            #res=snt.predict_sentence(content)
            
            #Using pipeline from transformers
            #res=sa([content])[0]["label"]
            
            labels=["positive","negative","neutral"]
            tokens = tokenizer.encode(content, return_tensors='pt')
            result = model(tokens)
            res=labels[int(torch.argmax(result.logits,dim=-1))]
            return res
            
        

In \[104\]:

    senti=Arabic_Sentiment()

In \[105\]:

    df=pd.read_csv("arabic.csv")

In \[106\]:

    pre=Arabic_Preprocessing()
    df.tweet=df.tweet.apply(pre.cleaning_arabic_content)

In \[111\]:

    for e,i in enumerate(df.tweet[:10]):
        print(e,i)

    0 طب وو وواوروبا كانو بيحرقو فلوس يااستاذ حليم
    1 انا وسطي واليسار اللبناني التكفيري يللي بنتقد تطرفه ما خصه اصلا باليسار
    2 اللهم تقبل منا ومنكم
    3 انا لا فرح لا حزن بصراحه كنتمني نبكي ولكن للاسف لا استطيع
    4 تاني سنه يتخالف فيها رؤيه الهلال مع مكه اذا العيد عندكم غير عندنا وعرفه عندكم غير عرفه عندنا والحجيج المغاربه سيقفوا علي عرفه في يوم ليس عندهم عرفه
    5 ويوم عرفه في السعوديه الحجاج يقفون بعرفه يوم الجمعه والعيد يوم السبت المغرب يوم عرفه هو يوم السبت وعرفه مافيه حتواحد
    6 تحليل فني لسهم تسلا يتوقع اختبار الدعم عند دولار
    7 هاي الكيكه من عدنا الك ايلون بلكي تشتري دوج كوين بمناسبه عيد ميلادك
    8 عطاك الله الصحه
    9 ليس كل ما يقوله وحيا افكاره من اسباب ثوره قوس قزح الهمجيه

In \[112\]:

    df["Arab_Sentiment"]=df.tweet.apply(senti.arabic_sentiment_analysis)

In \[115\]:

    df.Arab_Sentiment=df.Arab_Sentiment.map(lambda x: x.capitalize())
    df.Arab_Sentiment.value_counts()

Out\[115\]:

    Negative    52
    Positive    37
    Neutral     29
    Name: Arab_Sentiment, dtype: int64